<a href="https://colab.research.google.com/github/Dani3113R0se/SoDA/blob/main/rep_vouk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation of the paper

## Data

In [1]:
# Install necessary packages
!pip install pandas requests bs4 lxml gdelt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.4/787.4 kB 12.4 MB/s eta 0:00:00


In [2]:
# Let's load the GPI data
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import BytesIO

# data download
URL = "https://github.com/Dani3113R0se/SoDA/raw/d1f409f211b698cf8aec715304efb3b87c0cee0b/replication_voukelatou/data_peace.dta"
response = requests.get(URL)
response.raise_for_status()  # Verify successful download

# load to pandas
with BytesIO(response.content) as f:
    gpi_df = pd.read_stata(f)

# do we have all data
print(gpi_df)

      year      country  peace     id
0     2008  Afghanistan  3.153    1.0
1     2008      Albania  1.247    2.0
2     2008      Algeria  2.111    3.0
3     2008       Angola  1.655    4.0
4     2008    Argentina  1.403    5.0
...    ...          ...    ...    ...
2603  2023    Venezuela  2.086  159.0
2604  2023      Vietnam  1.403  160.0
2605  2023        Yemen  3.486  161.0
2606  2023       Zambia  1.577  162.0
2607  2023     Zimbabwe  2.006  163.0

[2608 rows x 4 columns]


## GDELT data

In [ ]:
# Import GDELT data
import gdelt
import pandas as pd
from datetime import datetime, timedelta  # Add timedelta to your import
gd = gdelt.gdelt(version=2)

# date range 2020-2023
start_date = datetime.strptime('2020 Jan 01', '%Y %b %d') # Convert to datetime
end_date = datetime.strptime('2023 Dec 31', '%Y %b %d')   # Convert to datetime

# Define chunk size (e.g., process data month by month)
chunk_size = timedelta(days=30)

# Initialize an empty DataFrame to store aggregated results
aggregated_data = pd.DataFrame()

# loop per month
current_start_date = start_date
while current_start_date < end_date:
    # end date per month
    current_end_date = min(current_start_date + chunk_size, end_date)

    # format dates
    formatted_start_date = current_start_date.strftime('%Y %b %d')
    formatted_end_date = current_end_date.strftime('%Y %b %d')

    # Retrieve GDELT data for the current month
    events = gd.Search(
        [formatted_start_date, formatted_end_date],
        table='events',
        output='pd',
        normcols=True,
        coverage=True
    )

    # filter and aggregate data for the current month
    if not events.empty:
        filtered_events = events.query("actiongeocountrycode != ''")
        chunk_aggregated = (
            filtered_events.groupby(['actiongeocountrycode', 'monthyear', 'eventbasecode'], as_index=False)
            .agg(
                No_events=('eventbasecode', 'count'),
                Tone=('STone', 'sum'),
                Goldstein=('SGoldsteinScale', 'sum')
            )
        )

        # aggregated results to the df
        aggregated_data = pd.concat([aggregated_data, chunk_aggregated], ignore_index=True)

    # next chunk
    current_start_date = current_end_date

# final df
aggregated_data = aggregated_data.sort_values(['actiongeocountrycode', 'eventbasecode', 'monthyear'])
print(aggregated_data.head())

here


/usr/local/lib/python3.11/dist-packages/gdelt/dateFuncs.py:241: UserWarning: This query will download 2976 files, and likely exhaust your memory with possibly 10s of GBs of data in this single query.Hit Ctr-C to kill this query if you do not want to continue.
  warnings.warn(warnText)
/usr/local/lib/python3.11/dist-packages/gdelt/dateFuncs.py:241: UserWarning: This query will download 2976 files, and likely exhaust your memory with possibly 10s of GBs of data in this single query.Hit Ctr-C to kill this query if you do not want to continue.
  warnings.warn(warnText)


xfsjdksffd

In [ ]:
print(result)